In [1]:
import argparse
import sys
import os
import math

from Bio.PDB.PDBParser import PDBParser
from Bio.PDB.NACCESS import NACCESS_atomic
from Bio.PDB.NeighborSearch import NeighborSearch
from Bio.PDB.PDBIO import PDBIO, Select
import biobb_structure_checking
import biobb_structure_checking.constants as cts
from biobb_structure_checking.structure_checking import StructureChecking
base_dir_path=biobb_structure_checking.__path__[0]
args = cts.set_defaults(base_dir_path,{'notebook':True})

In [2]:
from Bio import PDB
from Bio.PDB import NeighborSearch

def get_interface_residues(structure, chain1_id, chain2_id, distance_cutoff):
    model = structure[0]

    # Extract atoms from the two specified chains
    chain1_atoms = [atom for atom in model[chain1_id].get_atoms()]
    chain2_atoms = [atom for atom in model[chain2_id].get_atoms()]

    # Create NeighborSearch objects for each chain
    ns_chain1 = NeighborSearch(chain1_atoms)
    ns_chain2 = NeighborSearch(chain2_atoms)

    # Identify interface residues by searching for neighbors in the opposite chain
    int_res_chain1 = set()
    for atom in chain1_atoms:
        neighbors_chain2 = ns_chain2.search(atom.coord, distance_cutoff)
        if neighbors_chain2:
            int_res_chain1.add(atom.get_full_id()[3][1])

    int_res_chain2 = set()
    for atom in chain2_atoms:
        neighbors_chain1 = ns_chain1.search(atom.coord, distance_cutoff)
        if neighbors_chain1:
            int_res_chain2.add(atom.get_full_id()[3][1])

    return list(int_res_chain1), list(int_res_chain2)

IMPORTANT: Change the pathway for your own

In [3]:
# load the structure 
pdb_parser = PDB.PDBParser(QUIET=True)
structure = pdb_parser.get_structure("example", "/home/irene/Documents/BIOPYISICS/step_2/6m0j_fixed.pdb")

chain1_id = "A"
chain2_id = "E"
distance = 5

# output
int_res_chain1, int_res_chain2 = get_interface_residues(structure, chain1_id, chain2_id, distance)

print(f"Interface residues in Chain {chain1_id}: {int_res_chain1}")
print(f"Interface residues in Chain {chain2_id}: {int_res_chain2}")

Interface residues in Chain A: [386, 387, 393, 19, 24, 27, 28, 30, 31, 34, 35, 37, 38, 41, 42, 45, 324, 325, 326, 330, 79, 82, 83, 351, 352, 353, 354, 355, 357]
Interface residues in Chain E: [403, 417, 445, 446, 447, 449, 453, 455, 456, 712, 473, 475, 476, 477, 484, 486, 487, 489, 490, 493, 495, 496, 497, 498, 499, 500, 501, 502, 503, 504, 505]


SAME CODE AS ABOVE BUT THIS IS FROM THE GITHUB

In [4]:
def get_interface(st, dist):
    ''' Detects interface residues within a distance(dist)
        Assumes two chains, i.e. a unique interface set per chain.
    '''
    select_ats = []
    for at in st.get_atoms():
        # Skip Hydrogens to reduce time
        if at.element != 'H':
            select_ats.append(at)
    nbsearch = NeighborSearch(select_ats)
    interface = {}
    # Sets are more efficient than lists. Use sets when order is not relevant
    for ch in st[0]:
        interface[ch.id] = set()

    for at1, at2 in nbsearch.search_all(dist):
        #Only different chains
        res1 = at1.get_parent()
        ch1 = res1.get_parent()
        res2 = at2.get_parent()
        ch2 = res2.get_parent()
        if ch1 != ch2:
            interface[ch1.id].add(res1)
            interface[ch2.id].add(res2)
    return interface

IMPORTANT: Change the pathway for your own

In [5]:
pdb_path = '/home/irene/Documents/BIOPYISICS/step_2/6m0j_fixed.pdb'
parser = PDBParser(PERMISSIVE=1)
distance = 5
st = parser.get_structure('st', pdb_path)

In [6]:
get_interface(st, distance)

{'A': {<Residue SER het=  resseq=19 icode= >,
  <Residue GLN het=  resseq=24 icode= >,
  <Residue THR het=  resseq=27 icode= >,
  <Residue PHE het=  resseq=28 icode= >,
  <Residue ASP het=  resseq=30 icode= >,
  <Residue LYS het=  resseq=31 icode= >,
  <Residue HIE het=  resseq=34 icode= >,
  <Residue GLU het=  resseq=35 icode= >,
  <Residue GLU het=  resseq=37 icode= >,
  <Residue ASP het=  resseq=38 icode= >,
  <Residue TYR het=  resseq=41 icode= >,
  <Residue GLN het=  resseq=42 icode= >,
  <Residue LEU het=  resseq=45 icode= >,
  <Residue LEU het=  resseq=79 icode= >,
  <Residue MET het=  resseq=82 icode= >,
  <Residue TYR het=  resseq=83 icode= >,
  <Residue THR het=  resseq=324 icode= >,
  <Residue GLN het=  resseq=325 icode= >,
  <Residue GLY het=  resseq=326 icode= >,
  <Residue ASN het=  resseq=330 icode= >,
  <Residue LYS het=  resseq=353 icode= >,
  <Residue GLY het=  resseq=354 icode= >,
  <Residue ASP het=  resseq=355 icode= >,
  <Residue ARG het=  resseq=357 icode= >,
  <